Build a basic chatbot with langgraph using graph api

In [1]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [2]:
class State(TypedDict):
    message: Annotated[list, add_messages]

In [3]:
graph_builder = StateGraph(State)
graph_builder

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()